In [52]:
import csv
import numpy as np

In [53]:
fname = 'me_at_the_zoo.in'

In [54]:
ans = open(fname).read().splitlines()

## Parameters:

In [55]:
video_num, endpoint_num, request_num, cache_num, cache_size_limit = map(int, ans[0].split())

In [56]:
video_sizes = map(int, ans[1].split())

## Endpoints

In [57]:
rest = ans[2:]
endpoints = []
point = 0
for endpoint in range(endpoint_num):
    end_latent, num_connected = map(int,rest[point].split())
    caches = []
    for l in range(num_connected):
        caches.append(map(int,rest[point+l+1].split()))
    endpoints.append([end_latent, num_connected, caches])
    point += 1 + num_connected

## Requests

In [58]:
rest = rest[point:]

requests = []
for l in range(request_num):
    requests.append(map(int, rest[l].split()))